<a href="https://colab.research.google.com/github/meilongzhang/neurocodet5/blob/master/codet5_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install -q pytorch-lightning wandb

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 77 kB 7.6 MB/s 
     |████████████████████████████████| 895 kB 67.8 MB/s 
     |████████████████████████████████| 6.6 MB 73.9 MB/s 
     |████████████████████████████████| 596 kB 80.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 75.5 MB/s 
     |████████████████████████████████| 212 kB 98.2 MB/s 
     |████████████████████████████████| 136 kB 77.9 MB/s 
     |████████████████████████████████| 127 kB 69.3 MB/s 
     |████████████████████████████████| 94 kB 4.0 MB/s 
     |████████████████████████████████| 144 kB 92.3 MB/s 
     |████████████████████████████████| 271 kB 73.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
import pandas as pd
import json
from transformers import RobertaTokenizer, T5ForConditionalGeneration, TFT5ForConditionalGeneration

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
#model = TFT5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

Downloading:   0%|          | 0.00/687k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
text = "Generate Python: write a function that factorizes an integer"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
# this prints "{user.username}"

.


In [ ]:
data = pd.read_json('./lotlib_tune.json')

In [ ]:
''' Check to see what the longest entries in each column are after tokenizing
'''
code_token_lens = []
prompt_token_lens = []

for idx, row in data.iterrows():
    code, prompt = row
    code_ids = tokenizer(code, return_tensors="pt").input_ids
    code_token_lens.append(code_ids.shape[1])
    prompt_ids = tokenizer(prompt, return_tensors="pt").input_ids
    prompt_token_lens.append(prompt_ids.shape[1])

print(max(code_token_lens), max(prompt_token_lens))

57 237


In [ ]:
''' Splitting dataset into 0.9 train, 0.1 test and put it into HF datasets format
'''
from datasets import Dataset, DatasetDict

dataset = Dataset.from_pandas(data)
train_dataset, test_dataset= dataset.train_test_split(test_size=0.1).values()
train_dataset, validation_dataset = train_dataset.train_test_split(test_size=0.1).values()
datasets = DatasetDict({"train":train_dataset,"test":test_dataset, "validation":validation_dataset})

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['code', 'n1'],
        num_rows: 1209
    })
    test: Dataset({
        features: ['code', 'n1'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['code', 'n1'],
        num_rows: 135
    })
})

In [ ]:
''' Function to create the tokenized training examples
'''

prefix = "Generate Python: "
max_input_length = 512
max_target_length = 512

def preprocess_examples(examples):
     # encode the code-docstring pairs
    codes = examples['code']
    prompts = examples['n1']
    
    inputs = [prefix + prompt for prompt in prompts]
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)

    # encode the summaries
    labels = tokenizer(codes, max_length=max_target_length, padding="max_length", truncation=True).input_ids

    # important: we need to replace the index of the padding tokens by -100
    # such that they are not taken into account by the CrossEntropyLoss
    labels_with_ignore_index = []
    for labels_example in labels:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
        
    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs

In [ ]:
datasets = datasets.map(preprocess_examples, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['code', 'n1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1209
    })
    test: Dataset({
        features: ['code', 'n1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['code', 'n1', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 135
    })
})

In [ ]:
''' TODO: figure out how to interface with some training API, either PyTorch or Huggingface trainer
'''

' TODO: figure out how to interface with some training API, either PyTorch or Huggingface trainer\n'

In [ ]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer, Seq2SeqTrainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = datasets['train'],
    eval_dataset = datasets['test'],
    data_collator = data_collator,
    tokenizer = tokenizer,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: code, n1. If code, n1 are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1344
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 420


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=420, training_loss=0.499280766078404, metrics={'train_runtime': 17509.3336, 'train_samples_per_second': 0.384, 'train_steps_per_second': 0.024, 'total_flos': 909496905891840.0, 'train_loss': 0.499280766078404, 'epoch': 5.0})

In [ ]:
datasets['train']['n1'][0]

"Create a function that takes in each item in [{'shape': 'circle', 'color': 'blue'}, {'shape': 'triangle', 'color': 'red'}, {'shape': 'square', 'color': 'blue'}, {'shape': 'square', 'color': 'green'}] and returns the corresponding item in [True, False, True, True]"

In [ ]:
sample_output = model.generate(tokenizer(datasets['train']['n1'][0], return_tensors="pt").input_ids)
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

lambda x: (x['color']=='blue' or (not (x['color


In [ ]:
print(datasets['train']['code'][0])

lambda x: (x['shape']=='square' or not (x['color']=='red'))


In [ ]:
a = tokenizer(datasets['train']['code'][0], max_length=max_target_length, padding="max_length", truncation=True).input_ids

In [ ]:
tokenizer.decode(a, skip_special_tokens=True)

"lambda x: (x['shape']=='square' or not (x['color']=='red'))"

In [ ]:
""" Training using Pytorch
"""

' Training using Pytorch\n'

In [ ]:
from transformers import T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class CodeT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=15, warmup_steps=1000):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
import wandb

wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
model = CodeT5()

loading configuration file https://huggingface.co/Salesforce/codet5-small/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ef13e715cbf36adda46c74774e8032ab573cfbb2ebe59748c9fc72b7cf67e418.96d28e790b8c3d3e3be663606a66e0793a173c78e745e3603be4c0f878319099
Model config T5Config {
  "_name_or_path": "/content/drive/MyDrive/CodeT5/pretrained_models/codet5_small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "bos_token_id": 1,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 2,
  "feed_forward_proj": "relu",
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_m

In [ ]:
''' specific to pytorch, taken from https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/T5/Fine_tune_CodeT5_for_generating_docstrings_from_Ruby_code.ipynb
'''
from torch.utils.data import DataLoader

datasets.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])
train_dataloader = DataLoader(datasets['train'], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(datasets['validation'], batch_size=4)
test_dataloader = DataLoader(datasets['test'], batch_size=4)

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

wandb_logger = WandbLogger(name='codet5-finetune-code-summarization-ruby-shuffle', project='CodeT5')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(gpus=1, 
                  default_root_dir="/content/drive/MyDrive/CodeT5/Notebooks/Checkpoints", 
                  logger=wandb_logger, 
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:348: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /content/drive/MyDrive/CodeT5/Notebooks/Checkpoints/CodeT5/1qm0mx2j/checkpoints exists and is not empty.
  

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
save_directory = "."
model.model.save_pretrained(save_directory)

Configuration saved in ./config.json
Model weights saved in ./pytorch_model.bin


In [ ]:
test_example = test_dataset[99]
print("Prompt:", test_example['n1'])

Prompt: Create a function that takes in each item in [{'shape': 'circle', 'color': 'red'}, {'shape': 'square', 'color': 'green'}, {'shape': 'triangle', 'color': 'green'}, {'shape': 'circle', 'color': 'green'}] and returns the corresponding item in [True, False, False, True]


In [ ]:
print("Actual Code:", test_example['code'])

Actual Code: lambda x: (x['shape']=='circle' and not (x['shape']=='triangle'))


'''
Write a Python function that takes an array of dictionaries as input and returns True or False for each item according to its values. Your code should satisfy these tests:

assert my_function([{'shape': 'square', 'color': 'blue'}, {'shape': 'circle', 'color': 'green'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'circle', 'color': 'blue'}]) == [False, True, True, True]
assert my_function([{'shape': 'triangle', 'color': 'green'}, {'shape': 'circle', 'color': 'blue'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'square', 'color': 'green'}]) == [False, True, True, False]
assert my_function([{'shape': 'square', 'color': 'red'}, {'shape': 'triangle', 'color': 'red'}]) == [True, True]
'''

def my_function(arr):
    return [(x['shape']=='circle' or x['color']=='red') for x in arr]

'''
Write a Python function that takes an array of dictionaries as input and returns True or False for each item according to its values. Your code should satisfy these tests:

assert my_function([{'shape': 'square', 'color': 'green'}, {'shape': 'square', 'color': 'red'}, {'shape': 'circle', 'color': 'blue'}, {'shape': 'triangle', 'color': 'red'}]) == [True, True, True, True]
assert my_function([{'shape': 'circle', 'color': 'green'}, {'shape': 'square', 'color': 'blue'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'circle', 'color': 'blue'}]) == [False, True, True, True]
assert my_function([{'shape': 'triangle', 'color': 'blue'}, {'shape': 'square', 'color': 'red'}]) == [True, True]
'''

def my_function(arr):
    return [(not (x['color']=='green') or not (x['shape']=='circle')) for x in arr]
    
'''
Write a Python function that takes an array of dictionaries as input and returns True or False for each item according to its values. Your code should satisfy these tests:

assert my_function([{'shape': 'square', 'color': 'red'}, {'shape': 'triangle', 'color': 'green'}, {'shape': 'square', 'color': 'blue'}, {'shape': 'square', 'color': 'green'}]) == [True, True, True, True]
assert my_function([{'shape': 'square', 'color': 'red'}, {'shape': 'circle', 'color': 'green'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'square', 'color': 'blue'}]) == [True, True, True, True]
assert my_function([{'shape': 'triangle', 'color': 'green'}, {'shape': 'circle', 'color': 'green'}]) == [True, True]
'''

def my_function(arr):
    return [(not (x['color']=='red') or not (x['color']=='green')) for x in arr]
    
'''
Write a Python function that takes an array of dictionaries as input and returns True or False for each item according to its values. Your code should satisfy these tests:

assert my_function([{'shape': 'square', 'color': 'blue'}, {'shape': 'square', 'color': 'blue'}]) == [False, False]
assert my_function([{'shape': 'square', 'color': 'red'}, {'shape': 'square', 'color': 'blue'}]) == [True, False]
assert my_function([{'shape': 'square', 'color': 'green'}, {'shape': 'circle', 'color': 'blue'}]) == [True, False]
'''

def my_function(arr):
    return [(x['shape']=='triangle' or not (x['color']=='blue')) for x in arr]
    
'''
Write a Python function that takes an array of dictionaries as input and returns True or False for each item according to its values. Your code should satisfy these tests:

assert my_function([{'shape': 'circle', 'color': 'red'}, {'shape': 'square', 'color': 'green'}]) == [True, False]
assert my_function([{'shape': 'triangle', 'color': 'green'}, {'shape': 'circle', 'color': 'green'}]) == [False, True]
'''



**THIS ACTUALLY WORKED WOO**

'''
Write a Python function that takes a dictionary as input and returns True or False according to its values. Your code should satisfy these tests:

assert my_function({'shape': 'square', 'color': 'blue'}) == False
assert my_function({'shape': 'circle', 'color': 'green'}) == True
assert my_function({'shape': 'circle', 'color': 'red'}) == True
assert my_function({'shape': 'circle', 'color': 'blue'}) == True
assert my_function({'shape': 'triangle', 'color': 'green'}) == False
assert my_function({'shape': 'circle', 'color': 'blue'}) == True
assert my_function({'shape': 'circle', 'color': 'red'}) == True
assert my_function({'shape': 'square', 'color': 'green'}) == False
assert my_function({'shape': 'square', 'color': 'red'}) == True
assert my_function({'shape': 'triangle', 'color': 'red'}) == True
'''

def my_function(x):
    return (x['shape']=='circle' or x['color']=='red')

'''
Write a Python function that takes a dictionary as input and returns True or False according to its values. Your code should satisfy these tests:

assert my_function({'shape': 'square', 'color': 'green'}) == True
assert my_function({'shape': 'square', 'color': 'red'}) == True
assert my_function({'shape': 'circle', 'color': 'blue'}) == True
assert my_function({'shape': 'triangle', 'color': 'red'}) == True
assert my_function({'shape': 'circle', 'color': 'green'}) == False
assert my_function({'shape': 'square', 'color': 'blue'}) == True
assert my_function({'shape': 'circle', 'color': 'red'}) == True
assert my_function({'shape': 'circle', 'color': 'blue'}) == True
assert my_function({'shape': 'triangle', 'color': 'blue'}) == True
assert my_function({'shape': 'square', 'color': 'red'}) == True
'''

def my_function(x):
    return (not (x['color']=='green') or not (x['shape']=='circle'))
        
'''
Write a Python function that takes a dictionary as input and returns True or False according to its values. Your code should satisfy these tests:

assert my_function({'shape': 'square', 'color': 'red'}) == True
assert my_function({'shape': 'triangle', 'color': 'green'}) == True
assert my_function({'shape': 'square', 'color': 'blue'}) == True
assert my_function({'shape': 'square', 'color': 'green'}) == True
assert my_function({'shape': 'square', 'color': 'red'}) == True
assert my_function({'shape': 'circle', 'color': 'green'}) == True
assert my_function({'shape': 'circle', 'color': 'red'}) == True
assert my_function({'shape': 'square', 'color': 'blue'}) == True
assert my_function({'shape': 'triangle', 'color': 'green'}) == True
assert my_function({'shape': 'circle', 'color': 'green'}) == True
'''

def my_function(x):
    return (not (x['color']=='red') or not (x['color']=='green'))
    
'''
Write a Python function that takes a dictionary as input and returns True or False according to its values. Your code should satisfy these tests:

assert my_function({'shape': 'square', 'color': 'blue'}) == False
assert my_function({'shape': 'square', 'color': 'blue'}) == False
assert my_function({'shape': 'square', 'color': 'red'}) == True
assert my_function({'shape': 'square', 'color': 'blue'}) == False
assert my_function({'shape': 'square', 'color': 'green'}) == True
assert my_function({'shape': 'circle', 'color': 'blue'}) == False
'''

def my_function(x):
    return (x['shape']=='triangle' or not (x['color']=='blue'))
    
'''
Write a Python function that takes a dictionary as input and returns True or False according to its values. Your code should satisfy these tests:

assert my_function({'shape': 'circle', 'color': 'red'}) == True
assert my_function({'shape': 'square', 'color': 'green'}) == False
assert my_function({'shape': 'triangle', 'color': 'green'}) == False
assert my_function({'shape': 'circle', 'color': 'green'}) == True

'''

def my_function(x):
    return (x['shape']=='circle')
    
'''
Write a Python function that takes a dictionary as input and returns True or False according to its values. Your code should satisfy these tests:

assert my_function({'shape': 'square', 'color': 'red'}) == True
assert my_function({'shape': 'square', 'color': 'green'}) == False
assert my_function({'shape': 'triangle', 'color': 'red'}) == False
assert my_function({'shape': 'triangle', 'color': 'green'}) == True
assert my_function({'shape': 'circle', 'color': 'green'}) == False
'''


def my_function(x):
    return (x['shape']=='triangle' or x['color']=='red')

Prompt: Create a function called `my_function` that takes `data` as input and returns `output_values`. 

data = [{'shape': 'square', 'color': 'blue'}, {'shape': 'circle', 'color': 'green'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'circle', 'color': 'blue'}, {'shape': 'triangle', 'color': 'green'}, {'shape': 'circle', 'color': 'blue'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'square', 'color': 'green'}, {'shape': 'square', 'color': 'red'}, {'shape': 'triangle', 'color': 'red'}] 

output_values = [False, True, True, True, False, True, True, False, True, True]

my_function(data) == output_values

my_function = lambda data: [(x['shape']=='circle' or x['color']=='red') for x in data]
my_function(test_input)
[False, True, True, True, False, True, True, False, True, True]




In [ ]:
z = lambda input: [(x['shape']=='circle' or x['color']=='red') for x in input]
test_input = [{'shape': 'square', 'color': 'blue'}, {'shape': 'circle', 'color': 'green'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'circle', 'color': 'blue'}, {'shape': 'triangle', 'color': 'green'}, {'shape': 'circle', 'color': 'blue'}, {'shape': 'circle', 'color': 'red'}, {'shape': 'square', 'color': 'green'}, {'shape': 'square', 'color': 'red'}, {'shape': 'triangle', 'color': 'red'}] 
z(test_input)

[False, True, True, True, False, True, True, False, True, True]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(save_directory)
input_ids = tokenizer(test_example['n1'], return_tensors='pt').input_ids
outputs = model.generate(input_ids = input_ids, min_length=1, max_length=512)
output = tokenizer.decode(outputs[0], skip_special_tokens=True)
output_code = eval(output)
print("Generated Code:", output)

NameError: ignored

In [ ]:
print("Actual fn:", test_example['code'])

Actual fn: lambda x: (not (x['color']=='green') or not (x['shape']=='circle'))


In [ ]:
output_code({'shape':'square', 'color':'blue'})

True

In [ ]:
actual_code = eval(test_example['code'])

In [ ]:
actual_code({'shape':'square', 'color':'blue'})

False

Notes for feature engineering

1. Single feature, ie Green
2. Combo feature, ie Red Square
3. Negation + Or, ie (Blue or (Not Triangle))
4. Long combination ie (Green and Circle) or (Not Green and Square)